In [1]:
import cv2
import numpy as np

In [2]:
logo = cv2.imread('opencv-logo.png')

### 2D卷积操作（blur模糊操作）

In [17]:
kernel_identity = np.array([[0,0,0],[0,1,0],[0,0,0,]])
kernel_3x3 = np.ones((3,3), np.float32) / 9.0
kernel_5x5 = np.ones((5,5), np.float32) /25.0
kernel_7x7 = np.ones((5,5), np.float32) /25.0

In [20]:
#filter2D -1代表卷积深度，表示与源图像相同
identity = cv2.filter2D(logo,-1,kernel_identity)
img_3x3 = cv2.filter2D(logo, -1, kernel_3x3)
img_5x5 = cv2.filter2D(logo, -1, kernel_5x5)
img_7x7 = cv2.filter2D(logo, -1, kernel_7x7)
cv2.imshow('orginal', logo)
cv2.imshow('identity', identity)
cv2.imshow('3x3', img_3x3)
cv2.imshow('5x5', img_5x5)
cv2.imshow('7x7', img_7x7)
cv2.waitKey(0)
cv2.destroyAllWindows()

上面几个卷积核,kernel_identity是一个3x3，中心为1，其余8个点都为0卷积核心，它不会影响到图像数据。而后面几个全为1的卷积核，会平均化周围的数值，形成模糊效果。卷积核越大，模糊的范围也越大。这种将图像造成模糊的方法，有什么作用呢？一个很大的作用就是，去噪声。

In [21]:
## opencv提供了模糊卷积的功能，所以我们没有必要自己编写卷积核
output = cv2.blur(logo,(3,3))
cv2.imshow('blur',output)
cv2.imshow('img_3x3',img_3x3)#这两个效果是一样的
cv2.waitKey(0)
cv2.destroyAllWindows()

In [22]:
help(cv2.GaussianBlur)

Help on built-in function GaussianBlur:

GaussianBlur(...)
    GaussianBlur(src, ksize, sigmaX[, dst[, sigmaY[, borderType]]]) -> dst
    .   @brief Blurs an image using a Gaussian filter.
    .   
    .   The function convolves the source image with the specified Gaussian kernel. In-place filtering is
    .   supported.
    .   
    .   @param src input image; the image can have any number of channels, which are processed
    .   independently, but the depth should be CV_8U, CV_16U, CV_16S, CV_32F or CV_64F.
    .   @param dst output image of the same size and type as src.
    .   @param ksize Gaussian kernel size. ksize.width and ksize.height can differ but they both must be
    .   positive and odd. Or, they can be zero's and then they are computed from sigma.
    .   @param sigmaX Gaussian kernel standard deviation in X direction.
    .   @param sigmaY Gaussian kernel standard deviation in Y direction; if sigmaY is zero, it is set to be
    .   equal to sigmaX, if both sigmas are z

In [28]:
gaussian = cv2.GaussianBlur(logo,(5,5),1,1)
cv2.imshow('GaussianBlur',gaussian)
cv2.waitKey(0)
cv2.destroyAllWindows()

高斯模糊的思想与上面的不一样，上面的卷积核各个核的权重都是相等的（都是1）。而高斯模糊就不是这个样子，离卷积中心越远，其权重就越低，当然整个权重之和还是1.

### 边缘检测卷积核

In [8]:
Sx = np.array([[-1,0,1],[-2,0,2],[-1,0,1]])
Sy = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

In [9]:
s_x = cv2.filter2D(logo,-1,Sx)
s_y = cv2.filter2D(logo,-1,Sy)

In [10]:
cv2.imshow('Original', logo)
cv2.imshow('S_X', s_x)
cv2.imshow('S_Y', s_y)
cv2.waitKey(0)
cv2.destroyAllWindows()

显然上面的卷积不能使我们得到完整的边缘检测

In [11]:
#在opencv中可以直接调用Sobel算子
img = cv2.cvtColor(logo,cv2.COLOR_BGR2GRAY)
sobel_x = cv2.Sobel(img, cv2.CV_64F,1,0,ksize=3)
sobel_y = cv2.Sobel(img, cv2.CV_64F,0,1,ksize=3)

cv2.imshow('Original', img)
cv2.imshow('Sobel x', sobel_x)
cv2.imshow('Sobel y', sobel_y)
cv2.waitKey(0)
cv2.destroyAllWindows()
##这个效果要比上面好多了,但是依然不行

Sobel算子的缺陷还是很明显的，只能在水平和垂直方向分别检测到边缘，使用Laplacian能很好的克服这个缺点

In [13]:
laplacian = cv2.Laplacian(img, cv2.CV_64F)
cv2.imshow('laplacian',laplacian)
cv2.waitKey(0)
cv2.destroyAllWindows()

laplacian算子对于简单的形状效果不错，但是对于复杂的目标，其存在边缘检测细节过多的问题。Canny算子质量会更好。Canny是基于梯度的，它由两个参数，一个高梯度，一个低梯度。当梯度值大于高梯度时，就标记为边缘（强），然后追踪梯度小于低梯度的边。但这些值越大时，梯度小的边将会抛弃。


In [16]:
canny  = cv2.Canny(img, 100, 240)
cv2.imshow('canny',canny)
cv2.imshow('laplacian',laplacian)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [28]:
girl = cv2.imread('girl.jpg')
girl = cv2.cvtColor(girl,cv2.COLOR_BGR2GRAY)
girl_edge = cv2.Canny(girl,30,100)#增加细节
#girl_laplacian = cv2.Laplacian(girl, cv2.CV_64F)
girl_edge100 = cv2.Canny(girl,100,200)
cv2.imshow('girl',girl)
#cv2.imshow('girl_laplacian',girl_laplacian)
cv2.imshow('girl_edge',girl_edge)
cv2.imshow('girl_edge100',girl_edge100)
cv2.waitKey(0)
cv2.destroyAllWindows()